In [1]:
import copy
import json
import os
import sys
import time

In [2]:
from icecream import ic
import numpy as np
import pandas as pd
from pprint import pprint

import torch
import torch.multiprocessing as mp
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.models import densenet

from tqdm import tqdm

In [3]:
os.chdir('../')
sys.path.append('../dataset/')
import coco_data_prep
import coco_api_helper
import config_dataset

sys.path.append('../models/')
import densenet_inspired as di

loading annotations into memory...
Done (t=12.72s)
creating index...
index created!
loading annotations into memory...
Done (t=5.46s)
creating index...
index created!
loading annotations into memory...
Done (t=0.12s)
creating index...
index created!


In [4]:
%load_ext autoreload

### Global Variables

In [5]:
val_np_data_dir = '../data/numpy_imgs/val_subset/'
val_jpg_data_dir = '../data/raw/val/val2014/'
val_annot_filepath = '../data/raw/train/annotations/instances_val2014.json'

In [7]:
optim_params= dict(
    lr=0.001,
    betas=(0.9, 0.999),
    eps=1e-05,
    weight_decay=0.01,
    amsgrad=False,
)

### Load Checkpointed Model & Params

In [8]:
model = di.DensenetInspired()
optimizer = torch.optim.Adam(model.parameters(), **optim_params)

checkpoint = torch.load('./checkpoints/densenet_inspired_epoch_11.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

model.eval()

DensenetInspired(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (mp1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (conv2): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU(inplace=True)
  (conv3): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (flat): Flatten(start_dim=1, end_dim=-1)
  (lin1): LazyLinear(in_features=0, out_features=1024, bias=True)
  (fc1): Linear(in_features=1024, out_features=80, bias=True)
)

In [10]:
def slice_model(original_model, from_layer=None, to_layer=None):
    return nn.Sequential(*list(original_model.children())[from_layer:to_layer])

model_conv_features = slice_model(model, to_layer=-1).to('cpu')
model_conv_features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU()
  (6): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (8): ReLU(inplace=True)
  (9): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (10): Flatten(start_dim=1, end_dim=-1)
  (11): LazyLinear(in_features=0, out_features=1024, bias=True)
)

### Make Embeddings for all Val Dataset

In [11]:
all_val_ds = coco_data_prep.COCODataset('val',
                                        val_np_data_dir, 
                                        val_annot_filepath,
                                        coco_data_prep.COCOAnnotationTransform(),
                                        device='cpu')

loading annotations into memory...
Done (t=6.32s)
creating index...
index created!


100%|███████████████████████████████████████████| 40504/40504 [00:27<00:00, 1493.01it/s]


In [12]:
all_val_dl = coco_data_prep.get_dataloader(all_val_ds, 
                                         batch_size=512, 
                                         device='cpu')

In [ ]:
features_list = []
labels_list = []

# Use GPUs to speed up the inference, this should take around 10 minutes

model_conv_features.to('cpu')
for batch in tqdm(all_val_dl):
    image_batch, label_batch = [x[0] for x in batch], [x[1] for x in batch]
    image_batch = torch.stack(image_batch).to('cpu')

    with torch.no_grad():
        features_batch = model_conv_features(image_batch).flatten(start_dim=1)
        
    features_list.append(features_batch)
    labels_list.extend(label_batch)

  0%|                                                            | 0/42 [00:00<?, ?it/s]Exception ignored in sys.unraisablehookException ignored in sys.unraisablehook: Exception ignored in sys.unraisablehookException ignored in sys.unraisablehookException ignored in sys.unraisablehookException ignored in sys.unraisablehook: : <built-in function unraisablehook>: : Exception ignored in sys.unraisablehook<built-in function unraisablehook>Exception ignored in sys.unraisablehook<built-in function unraisablehook>
Traceback (most recent call last):
: : 
  File "/home/ec2-user/miniconda3/envs/cs7643-raml/lib/python3.8/site-packages/ipykernel/iostream.py", line 527, in write
<built-in function unraisablehook>
    <built-in function unraisablehook><built-in function unraisablehook>Traceback (most recent call last):
  File "/home/ec2-user/miniconda3/envs/cs7643-raml/lib/python3.8/site-packages/ipykernel/iostream.py", line 527, in write
Exception ignored in sys.unraisablehookTraceback (most recent


  File "/home/ec2-user/miniconda3/envs/cs7643-raml/lib/python3.8/site-packages/jupyter_client/session.py", line 753, in send
  File "/home/ec2-user/miniconda3/envs/cs7643-raml/lib/python3.8/site-packages/ipykernel/iostream.py", line 487, in _flush
            <built-in function unraisablehook>        : Traceback (most recent call last):
    
Exception ignored in sys.unraisablehook
f()Traceback (most recent call last):
: 

self.pub_thread.schedule(self._flush):   File "/home/ec2-user/miniconda3/envs/cs7643-raml/lib/python3.8/site-packages/ipykernel/iostream.py", line 216, in schedule


    Exception ignored in sys.unraisablehookTraceback (most recent call last):
    f()f()
self.session.send(self.pub_thread, 'stream', content=content,f()self.pub_thread.schedule(self._flush)  File "/home/ec2-user/miniconda3/envs/cs7643-raml/lib/python3.8/site-packages/ipykernel/iostream.py", line 527, in write
f()<built-in function unraisablehook>: 
      File "/home/ec2-user/miniconda3/envs/cs7643-raml/

f()  File "/home/ec2-user/miniconda3/envs/cs7643-raml/lib/python3.8/site-packages/ipykernel/iostream.py", line 487, in _flush


    
self.schedule(lambda : self._really_send(*args, **kwargs))      File "/home/ec2-user/miniconda3/envs/cs7643-raml/lib/python3.8/site-packages/jupyter_client/session.py", line 753, in send



  File "/home/ec2-user/miniconda3/envs/cs7643-raml/lib/python3.8/site-packages/ipykernel/iostream.py", line 216, in schedule
  File "/home/ec2-user/miniconda3/envs/cs7643-raml/lib/python3.8/site-packages/jupyter_client/session.py", line 753, in send


self.session.send(self.pub_thread, 'stream', content=content,            
    self.session.send(self.pub_thread, 'stream', content=content,    stream.send_multipart(to_send, copy=copy)self.session.send(self.pub_thread, 'stream', content=content,        
  File "/home/ec2-user/miniconda3/envs/cs7643-raml/lib/python3.8/site-packages/ipykernel/iostream.py", line 487, in _flush
  File "/home/ec2-user/miniconda3/envs/cs7643-ra